In [15]:
import pandas as pd
import clean_images
import data_cleaning
from sklearn.model_selection import train_test_split
import numpy as np
from numpy import asarray
import os
import torch
import torch.nn as nn
import image_text_dataset
import tensorboard_manager
import combined_model
from tqdm import tqdm

In [16]:
products = pd.read_csv("data/Products.csv",lineterminator='\n')
images = pd.read_csv("data/Images.csv",lineterminator='\n')

In [17]:
products.head()

,Unnamed: 0,id,product_name,category,product_description,price,location,url,page_id,create_time
0,0,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire",https://www.gumtree.com/p/mirrors-clocks-ornam...,1426704584,2022-02-26
1,1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...,1426704579,2022-02-26
2,2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,£20.00,"Skegness, Lincolnshire",https://www.gumtree.com/p/outdoor-settings-fur...,1426704576,2022-02-26
3,3,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset",https://www.gumtree.com/p/other-dining-living-...,1426704575,2022-02-26
4,4,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,£450.00,"Delph, Manchester",https://www.gumtree.com/p/sofas/modern-shannon...,1426704570,2022-02-26


In [18]:
products  = products.merge(images, how='inner',left_on='id',right_on='product_id')
products=products.rename(columns={"id_y":"image_id"})
products=products.rename(columns={"id_x":"id"})
products.drop('product_id',inplace=True,axis = 1)
products.drop('Unnamed: 0_y',inplace=True,axis = 1)
products.drop('url',inplace=True,axis = 1)
products.drop('page_id',inplace=True,axis = 1)
products.drop('create_time',inplace=True,axis = 1)
products.drop('bucket_link',inplace=True,axis = 1)
products.drop('image_ref',inplace=True,axis = 1)
products.drop('create_time\r',inplace=True,axis = 1)

In [19]:
products["product_name"] = products["product_name"].apply(lambda x: data_cleaning.clean_word(x)) 
products["product_description"] = products["product_description"].apply(lambda x: data_cleaning.clean_word(x)) 
products["location"] = products["location"].apply(lambda x: x.split(",")[1] if (len(x.split(",")) ==2) else x) 

In [20]:
cat= [ind for ind, x in enumerate(products["category"].unique())]
products["category"] = products["category"].apply(lambda x: data_cleaning.create_categories(x))
catToNumb = dict(zip(products["category"].unique(),cat ))
products["category"] = products["category"].apply(lambda x: data_cleaning.convert_cat_to_number(x,catToNumb))

In [21]:
num_categories = len(products.category.unique())

In [22]:
products["price"] = data_cleaning.clean_price(products["price"])

In [23]:
training,validation,test = image_text_dataset.split_train_test(products)
#clean_images.clean("data/images/","data/resized_images/",training)
#clean_images.clean("data/images/","data/resized_images/",validation)

In [24]:
training_dataset = image_text_dataset.ImageTextDataset(ds=training)
validation_dataset = image_text_dataset.ImageTextDataset(ds=validation)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


fatto


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


fatto


In [25]:
training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=12,
                                             shuffle=True, num_workers=1)

validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=12,
                                             shuffle=True, num_workers=0)

In [26]:
combined = combined_model.CombinedModel(ngpu=1, input_size=768, num_classes=training_dataset.num_classes)

Using cache found in C:\Users\alnar/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


In [27]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
combined.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(combined.parameters(), lr=0.001)
epochs = 5
for epoch in range(epochs):
    hist_acc = []
    hist_loss = []
    pbar = tqdm(enumerate(training_dataloader), total=len(training_dataloader))
    for i, (image_features, text_features, labels) in pbar:
        image_features = image_features.to(device)
        text_features = text_features.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = combined(image_features, text_features)
        loss = criterion(outputs, labels)
        loss.backward()
        hist_acc.append(torch.mean((torch.argmax(outputs, dim=1) == labels).float()).item())
        hist_loss.append(loss.item())
        optimizer.step()
        pbar.set_description(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item():.4f} Acc = {round(torch.sum(torch.argmax(outputs, dim=1) == labels).item()/len(labels), 2)} Total_acc = {round(np.mean(hist_acc), 2)}')
#torch.save(combined.state_dict(), 'combined_model.pt')

Epoch 5/5 Acc = 0.90: 100%|██████████| 5/5 [02:02<00:00, 24.59s/it]


In [28]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
combined.to(device)
criterion = nn.CrossEntropyLoss()
epochs = 5
for epoch in range(epochs):
    hist_acc = []
    hist_loss = []
    pbar = tqdm(enumerate(validation_dataloader), total=len(validation_dataloader))
    for i, (image_features, text_features, labels) in pbar:
        image_features = image_features.to(device)
        text_features = text_features.to(device)
        #print(max(labels))
        #print(min(labels))
        labels = labels.to(device)
        outputs = combined(image_features, text_features)
        loss = criterion(outputs, labels)
        hist_acc.append(torch.mean((torch.argmax(outputs, dim=1) == labels).float()).item())
        hist_loss.append(loss.item())
        pbar.set_description(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item():.4f} Acc = {round(torch.sum(torch.argmax(outputs, dim=1) == labels).item()/len(labels), 2)} Total_acc = {round(np.mean(hist_acc), 2)}')
#torch.save(combined.state_dict(), 'combined_model.pt')

Epoch 5/5 Acc = 0.85: 100%|██████████| 5/5 [00:20<00:00,  4.13s/it]
